# Notebook Contents

In this notebook the solar access lattice is calculated for the voxelated grid. 
The sun positions are calculated for the location of the site in Buiksloterham using the library https://www.ladybug.tools/ladybug/docs/_modules/ladybug/sunpath.html
The centroids for the voxels are found using the library https://topogenesis.readthedocs.io/
and intersection percentages are calulated with the context of the building massing and the site using the library https://trimsh.org/trimesh.html

# Initilization

In [1]:
import os
import sys
import topogenesis as tg
import pyvista as pv
import trimesh as tm
from itertools import cycle
import numpy as np
import pickle
import numpy.ma as ma
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
from ladybug.sunpath import Sunpath

In [2]:
#Design Details
Self_development_plots_path = os.path.relpath('Site_self_development_plots.obj')
Self_development_backyards_path = os.path.relpath('Site_self_development_backyards.obj')
Site_buildings_path = os.path.relpath('Site_buildings.obj')
Site_green_areas_path = os.path.relpath('Site_green_areas.obj')
Site_roads_path = os.path.relpath('Site_base_roads.obj')
Site_context_shading_path= os.path.relpath('Site_surrounding_buildings_for_shading.obj') 
Context_mesh_for_building_one_path = os.path.relpath('Context_for_building_one.obj') 
Context_mesh_for_building_two_and_three_path = os.path.relpath('Context_for_building_two_three.obj') 
Context_mesh_for_building_four_path = os.path.relpath('Context_for_building_four.obj') 

# Site details
Site_base_path = os.path.relpath('Site_base_block.obj')
Site_surrounding_buildings_path = os.path.relpath('Site_surrounding_buildings.obj')
Site_water_bodies_path = os.path.relpath('Site_water_bodies.obj')
Site_roads_path = os.path.relpath('Site_roads.obj')
Site_other_buildings_path = os.path.relpath('Site_other_buildings.obj')

# load the mesh from file
# Design elements
Self_development_plots_mesh = tm.load(Self_development_plots_path)
Self_development_backyards_mesh = tm.load(Self_development_backyards_path)
Site_building_mesh = tm.load(Site_buildings_path)
Site_green_areas_mesh = tm.load(Site_green_areas_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_context_shading_mesh = tm.load(Site_context_shading_path)
Context_mesh_for_building_one_mesh = tm.load(Context_mesh_for_building_one_path)
Context_mesh_for_building_two_and_three_mesh =tm.load(Context_mesh_for_building_two_and_three_path)
Context_mesh_for_building_four_mesh =tm.load(Context_mesh_for_building_four_path)

#Site elements
Site_base_mesh = tm.load(Site_base_path)
Site_surrounding_buildings_mesh = tm.load(Site_surrounding_buildings_path)
Site_water_bodies_mesh = tm.load(Site_water_bodies_path)
Site_roads_mesh = tm.load(Site_roads_path)
Site_other_buildings_mesh = tm.load(Site_other_buildings_path)


# Check if the mesh is watertight
#print(envelope_mesh.is_watertight)
#print(context_mesh.is_watertight)

faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!


In [3]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#f4acb7')
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')


# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1563.6562028391934, 1809.440031574545, 1616.2765183909512),
 (-142.0987091064453, 103.68511962890625, -89.4783935546875),
 (0.0, 0.0, 1.0)]

# Import lattice

In [4]:
# loading the lattice from csv
lattice_path = os.path.relpath('voxelized_envelope_6m_voxel_size.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

# Find Voxel Eucledian Coordinates

In [5]:
# convert to lattice
init_lattice = envelope_lattice +1
availability_lattice_voxels = tg.to_lattice(init_lattice, init_lattice)
voxel_coordinates= envelope_lattice.centroids
voxel_coordinates_full_lattice = init_lattice.centroids
flattened_lattice = envelope_lattice.flatten()

In [6]:
Y_coordinates= voxel_coordinates.T[1].flatten()

In [7]:
Full_lattice_Y_coordinates = voxel_coordinates_full_lattice.T[1].flatten()

In [8]:
Building_one =[]
Building_one_indexes =[]
Building_two_three =[]
Building_two_three_indexes =[]
Building_four =[]
Building_four_indexes =[]
not_in_a_building =[]
for center,coordinate,index in zip(Full_lattice_Y_coordinates,voxel_coordinates_full_lattice,range(len(Full_lattice_Y_coordinates))):
    if center >= 18 and center <= 30:
        Building_one.append(coordinate)
        Building_one_indexes.append(index)
        #print("1st")
    elif center >= 40 and center <= 82:
        Building_two_three.append(coordinate)
        Building_two_three_indexes.append(index)
       # print("2nd")
    elif center >= 82:
        Building_four.append(coordinate)
        Building_four_indexes.append(index)
        #print("3rd")
    else:
        not_in_a_building.append(coordinate)
    

In [9]:
Building_one_indexes

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 352,
 353,
 354,
 355,
 356,
 357,
 358,
 359,
 360,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 375,
 376,
 377,
 378,
 379,
 380,
 381,
 382,
 383,
 384,
 528,
 529,
 530,
 531,
 532,
 533,
 534,
 535,
 536,
 537,
 538,
 539,
 540,
 541,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 550,
 551,
 552,
 553,
 554,
 555,
 556,
 557,
 558,
 559,
 560,
 704,
 705,
 706,
 707,
 708,
 709,
 710,
 711,
 712,
 713,
 714,
 715,
 716,
 717,
 718,
 719,
 720,
 721,
 722,
 723,
 724,
 725,
 726,
 727,
 728,
 729,
 730,
 731,
 732,
 733,
 734,
 735,
 736,
 880,
 881,
 882,
 883,
 884,
 885,
 886,
 887,
 888,

# Visualize the Context Mesh + Envelope Lattice

In [10]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes

# Design meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1564.3134508410321, 1810.0972795763837, 1618.43376639279),
 (-142.0987091064453, 103.68511962890625, -87.9783935546875),
 (0.0, 0.0, 1.0)]

# Sunvectors

## Compute sun vectors

In [11]:
# initiate sunpath
sp = Sunpath(longitude=4.90, latitude=52.39)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
hoys = []
sun_vectors = []
day_multiples = 30
for d in range(365):
    if d%day_multiples==0:
        for h in range(24):
            i = d*24 + h
            # compute the sun object
            sun = sp.calculate_sun_from_hoy(i)
            # extract the sun vector
            sun_vector = sun.sun_vector.to_array()
            # apparantly, if the Z component of sun vector is positive, 
            # it is under the horizon 
            if sun_vector[2] < 0.0:
                hoys.append(i)
                sun_vectors.append(sun_vector)
                
sun_vectors = np.array(sun_vectors)
# compute the rotation matrix 
Rz = tm.transformations.rotation_matrix(np.radians(36.324), [0,0,1])
# Rotate the sun vectors to match the site rotation
sun_vectors = tm.transform_points(sun_vectors, Rz)
print(sun_vectors.shape)

(154, 3)


In [12]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_surrounding_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')

# add the sun locations, color orange
p.add_points( - sun_vectors * 300, color='#ffa500')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(1621.503249013592, 1867.2870777489436, 1774.9954138548158),
 (-142.0987091064453, 103.68511962890625, 11.393455734778485),
 (0.0, 0.0, 1.0)]

# Compute Intersection of Sun Rays with Context Mesh

### Preparing the List of Ray Directions and Origins

In [13]:
# constructing the sun direction from the sun vectors in a numpy array
sun_dirs = -np.array(sun_vectors)


# next step we need to shoot in all of the sun directions from all of the voxels, todo so, we need repeat the sun direction for the number of voxels to construct the ray_dir (which is the list of all ray directions). We need to repeat the voxels for the 

# Further info: this is the vectorised version of nested for loops
ray_dir_building_one = np.tile(sun_dirs, [len(Building_one),1])
ray_src_building_one = np.tile(Building_one, [1, len(sun_dirs)]).reshape(-1, 3)

ray_dir_building_two_and_three = np.tile(sun_dirs, [len(Building_two_three),1])
ray_src_building_two_and_three = np.tile(Building_two_three, [1, len(sun_dirs)]).reshape(-1, 3)

ray_dir_building_four = np.tile(sun_dirs, [len(Building_four),1])
ray_src_building_four = np.tile(Building_four, [1, len(sun_dirs)]).reshape(-1, 3)

#print("number of voxels to shoot rays from :",vox_cens.shape)
print("number of rays per each voxel :",sun_dirs.shape)
#print("number of rays to be shooted :",ray_src.shape)

number of rays per each voxel : (154, 3)


# Computing the Intersection

In [14]:
# computing the intersections of rays with the context mesh
tri_id_building_one, ray_id_building_one = Context_mesh_for_building_one_mesh.ray.intersects_id(ray_origins=ray_src_building_one, ray_directions=ray_dir_building_one, multiple_hits=False)
tri_id_building_two_and_three, ray_id_building_two_and_three = Context_mesh_for_building_two_and_three_mesh.ray.intersects_id(ray_origins=ray_src_building_two_and_three, ray_directions=ray_dir_building_two_and_three, multiple_hits=False)
tri_id_building_four, ray_id_building_four = Context_mesh_for_building_four_mesh.ray.intersects_id(ray_origins=ray_src_building_four, ray_directions=ray_dir_building_four, multiple_hits=False)


## Aggregate Simulation Result in the Sun Access Lattice

###  Compute the percentage of time that each voxel sees the sun

In [15]:
def percentage_of_time_each_voxel_sees_the_sun(ray_dir,ray_id,sun_dirs,vox_cens):
    # initializing the hits list full of zeros
    hits = [0]*len(ray_dir)
    # setting the rays that had an intersection to 1
    for id in ray_id:
        hits[id] = 1

    sun_count = len(sun_dirs)
    vox_count = len(vox_cens)
    # initiating the list of ratio
    vox_sun_acc = []
    # iterate over the voxels
    for v_id in range(vox_count):
        # counter for the intersection
        int_count = 0
        # iterate over the sun rays
        for s_id in range(sun_count):
            # computing the ray id from voxel id and sun id
            r_id = v_id * sun_count + s_id

            # summing the intersections
            int_count += hits[r_id]

        # computing the percentage of the rays that DID NOT have 
        # an intersection (aka could see the sun)
        sun_access = 1.0 - int_count/sun_count

        # add the ratio to list
        vox_sun_acc.append(sun_access)

    hits = np.array(hits)
    vox_sun_acc = np.array(vox_sun_acc)

    return hits , vox_sun_acc




In [16]:
percent_for_building_one= percentage_of_time_each_voxel_sees_the_sun(ray_dir_building_one,ray_id_building_one,sun_dirs,Building_one)
percent_for_building_two_and_three= percentage_of_time_each_voxel_sees_the_sun(ray_dir_building_two_and_three,ray_id_building_two_and_three,sun_dirs,Building_two_three)
percent_for_building_Four= percentage_of_time_each_voxel_sees_the_sun(ray_dir_building_four,ray_id_building_four,sun_dirs,Building_four)

### 5.2. Store sun access information in a Lattice

In [17]:
percent_for_building_Four[1][np.argmax(percent_for_building_Four[1])]

1.0

In [18]:
Dummy_lattice = np.copy(Full_lattice_Y_coordinates)

In [19]:
for index,value in zip(Building_one_indexes,percent_for_building_one[1]):
    Dummy_lattice[index]= value

In [20]:
for index,value in zip(Building_two_three_indexes,percent_for_building_two_and_three[1]):
    Dummy_lattice[index]= value

In [21]:
for index,value in zip(Building_four_indexes,percent_for_building_Four[1]):
    Dummy_lattice[index]= value

In [22]:
Solar_lattice_padded= np.array([num if boolean else 0 for boolean, num in zip(flattened_lattice, cycle(Dummy_lattice))])

In [23]:
Solar_lattice_padded_np =  Solar_lattice_padded.reshape(envelope_lattice.shape)

In [24]:
Solar_lattice = tg.to_lattice(Solar_lattice_padded_np, Solar_lattice_padded_np.shape)

# Vizualise the sun access lattice 

In [27]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid envelope_lattice: shape because we want to inject our values
grid.dimensions = envelope_lattice.shape
# The bottom left corner of the data set
grid.origin = envelope_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = envelope_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Solar Access"] = Solar_lattice.flatten(order="F")  # Flatten the Lattice

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
#p.add_mesh(tri_to_pv(Self_development_plots_mesh), color='#b8f2e6')
p.add_mesh(tri_to_pv(Self_development_backyards_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_building_mesh), color='#ff9b54',opacity = 0.3)
p.add_mesh(tri_to_pv(Site_green_areas_mesh), color='#8ac926')
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')

#Site meshes
p.add_mesh(tri_to_pv(Site_base_mesh), color='#faedcd')
p.add_mesh(tri_to_pv(Site_other_buildings_mesh), color='#cdb4db')
p.add_mesh(tri_to_pv(Site_water_bodies_mesh), color='#bde0fe',opacity= 0.5)
p.add_mesh(tri_to_pv(Site_roads_mesh), color='#adb5bd')
p.add_mesh(tri_to_pv(Site_context_shading_mesh), color='#cdb4db')
    
# adding the volume

opacity = [0, 0.75, 0.7, 0.75, 0.8]
clim = [0, 100]
p.add_volume(grid, cmap="inferno", opacity=opacity, shade=False)
# plotting
p.add_points( - sun_vectors * 300, color='#ffa500')
p.camera_position = [(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154929, -0.31327193009210785, 0.8220766014501246)]
p.show(use_ipyvtk=True,screenshot='Solar_lattice.png')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(281.2198164557486, 195.20681864151288, 263.2631846148646),
 (-125.74100344423854, 28.782304005903896, -35.52262026413212),
 (-0.4754479563154928, -0.3132719300921078, 0.8220766014501244)]

# Save Pickle Lattice

In [26]:
#Solar_access_mtrx = pickle.dump( Solar_lattice, open( "Solar_access_lattice.p", "wb" ) )